In [1]:
import time
import io
import pandas as pd
from pulp import *
import numpy as np


In [4]:
# Load data from schedule csv
try:
  data = pd.read_csv("F24-Schedule-ECE.csv")
except FileNotFoundError:
  print("File not found")

data.columns = data.columns.str.strip()
data['OFFERING_ID'] = data.index

In [5]:
all_instructors = set()
for inst_list in data['INSTRUCTOR'].astype(str).tolist():
  for inst_name in inst_list.split(';'):
    cleaned_name = inst_name.strip().replace('"', '')
    if cleaned_name:
      all_instructors.add(cleaned_name)

unique_instructors = list(all_instructors)

offerings = data['OFFERING_ID'].tolist()

data['UNITS'] = pd.to_numeric(data['UNITS'], errors='coerce').fillna(0)
unique_courses = data['COURSE'].unique().tolist()
unique_locations = data[~data['BLDG/ROOM'].isin(['TBA', 'DNM', 'TBD'])]['BLDG/ROOM'].unique().tolist()

print(f"Data loaded with {len(unique_instructors)} unique instructors.\n")
print(f"Data loaded with {len(data)} course offerings.\n")



Data loaded with 63 unique instructors.

Data loaded with 140 course offerings.



In [6]:
def time_to_minutes(time_str):
  # Converting HH:MMAM/PM to minutes from midnight (0 to 1439).
  if time_str in ('TBA', 'DNM', 'TBD'): return -1

  time_str = time_str.zfill(7) if len(time_str) == 6 else time_str

  try:
    dt = pd.to_datetime(time_str, format='%I:%M%p')
    return dt.hour * 60 + dt.minute
  except ValueError:
    if time_str.endswith('P'):
      return time_to_minutes(time_str + 'M')
    return -1

def check_overlap(start1_str, end1_str, start2_str, end2_str):
  start1, end1 = time_to_minutes(start1_str), time_to_minutes(end1_str)
  start2, end2 = time_to_minutes(start2_str), time_to_minutes(end2_str)
  if start1 == -1 or start2 == -1: return False
  return max(start1, start2) < min(end1, end2)


In [7]:
from enum import unique
import time

def solve_scheduling_ilp(data_subset):
  # Solves ILP for a given subset
  offerings = data_subset['OFFERING_ID'].tolist()
  subset_unique_courses = data_subset['COURSE'].unique().tolist()
  subset_unique_locations = data_subset[~data_subset['BLDG/ROOM'].isin(['TBA', 'DNM', 'TBD'])]['BLDG/ROOM'].unique().tolist()

  prob = LpProblem("Scheduling_Subset", LpMaximize)
  X = LpVariable.dicts("Select", offerings, 0, 1, LpBinary)

  prob += lpSum(data_subset.loc[i, 'UNITS'] * X[i] for i in offerings), "Maximize_Total_Units_Scheduled"

  for course_id in subset_unique_courses:
    course_offerings = data_subset[data_subset['COURSE'] == course_id]['OFFERING_ID'].tolist()
    prob += lpSum(X[i] for i in course_offerings) <= 1, f"C_Max_One_Section_of_{course_id}"

  for location in subset_unique_locations:
    location_offerings = data_subset[data_subset['BLDG/ROOM'] == location]['OFFERING_ID'].tolist()

    for i in location_offerings:
      for j in location_offerings:
        if i < j:
          course_i = data_subset.loc[i]
          course_j = data_subset.loc[j]

          days_i = set(course_i['DAYS'])
          days_j = set(course_j['DAYS'])
          common_days = days_i.intersection(days_j)

          if common_days and check_overlap(course_i['BEGIN'], course_i['END'], course_j['BEGIN'], course_j['END']):

            prob += X[i] + X[j] <= 1, f"RT_Room_Time_Conflict_{location}_{i}_{j}"

  start_time = time.time()
  prob.solve()
  runtime = time.time() - start_time

  status = LpStatus[prob.status]
  optimal_units = value(prob.objective) if status == 'Optimal' else 0
  num_constraints = len(prob.constraints)

  return {
        'N_Offerings': len(offerings),
        'N_Courses': len(subset_unique_courses),
        'N_Constraints': num_constraints,
        'Runtime_s': runtime,
        'Optimal_Units': optimal_units,
        'Status': status
    }

In [8]:

results_data = []
sample_sizes = np.linspace(10, len(data), 10, dtype=int)
print("Starting classical ILP solver (CBC) with C, T, R constraints...")

for size in sample_sizes:
  data_subset = data.iloc[:size].copy()

  data_subset['OFFERING_ID'] = range(len(data_subset))
  # data_subset.set_index('OFFERING_ID', inplace=True) # This line caused the KeyError by removing the 'OFFERING_ID' column.

  data_subset.loc[:, 'UNITS'] = pd.to_numeric(data_subset['UNITS'], errors='coerce').fillna(0)

  metrics = solve_scheduling_ilp(data_subset)
  results_data.append(metrics)
  print(f"Solved size {metrics['N_Offerings']}: Runtime {metrics['Runtime_s']:.4f}s")

Starting classical ILP solver (CBC) with C, T, R constraints...
Solved size 10: Runtime 0.0474s
Solved size 24: Runtime 0.0046s
Solved size 38: Runtime 0.0044s
Solved size 53: Runtime 0.0068s
Solved size 67: Runtime 0.0055s
Solved size 82: Runtime 0.0065s
Solved size 96: Runtime 0.0066s
Solved size 111: Runtime 0.0065s
Solved size 125: Runtime 0.0063s
Solved size 140: Runtime 0.0095s


In [9]:
results_csv = pd.DataFrame(results_data)
csv_output_filename = 'Classical_ILP_Performance_Data.csv'
results_csv.to_csv(csv_output_filename, index=False)


print(results_csv.head().to_markdown())

|    |   N_Offerings |   N_Courses |   N_Constraints |   Runtime_s |   Optimal_Units | Status   |
|---:|--------------:|------------:|----------------:|------------:|----------------:|:---------|
|  0 |            10 |           4 |               5 |  0.0473926  |              24 | Optimal  |
|  1 |            24 |           7 |               8 |  0.00460482 |              49 | Optimal  |
|  2 |            38 |           9 |              10 |  0.00444555 |              73 | Optimal  |
|  3 |            53 |          13 |              15 |  0.00680447 |             121 | Optimal  |
|  4 |            67 |          23 |              25 |  0.00551486 |             226 | Optimal  |
